### 크롤링 도중 ip 차단을 당했고 실시간으로 ip를 랜덤하게 변환시키는 방법을 찾음<br> 그러나 Gamespy에서 제공하는 api로 id와 name을 찾음<br> 시간상의 이유나 모든 상황을 볼때 새로 steam spy에서 크롤링하는것이 더 좋은 판단이라고 생각되어 크롤링을 새로 시작

# 딕셔너리 자료 변환

In [1]:
import time
import pandas as pd
import requests
import pymysql
import json
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
from selenium import webdriver

In [2]:
host_name = "127.0.0.1"
username = "root"
password = ""
database_name = "steam_test"

In [3]:
db = pymysql.connect(
    host = host_name,
    port=3306,
    user=username,
    passwd=password,
    db=database_name,
    charset='utf8'
)
cursor= db.cursor()

## 테이블 생성

In [4]:
start_time = time.time()
#-----------------------------------------------------------------------------------------
chromedriver = '/home/nincastle/chromedriver/chromedriver' # ubuntu
# chromedriver = 'D:\chromedriver_win32/chromedriver' # windows
tags_genre_query = []

def name_replace(name):
    name = name.replace("&","and").replace(".","dot")\
    .replace("Mod","Mod_").replace("'","").replace("-","_").replace("Linear","Linear_")\
    .replace("Action","Action_").replace("Foreign","Foreign_").replace("All","All_")
    return name

def driver_chrome(url, xpath_):
    driver_tags = webdriver.Chrome(chromedriver)
    driver_tags.get(url)
    print(type(xpath_))
    tags = driver_tags.find_element_by_xpath(xpath_)
    t_g = tags.find_elements_by_tag_name('a')
    driver_tags.quit
    return t_g
# 테그 변수 이름 크롤링
cro_tag = driver_chrome("https://steamdb.info/tags/","//div[@class='row list']")
for t in cro_tag:
    t = t.text.replace(" ","_")   
    tags_genre_query.append("tag_"+t+' INT(1)')


# 장르 변수 이름 크롤링
cro_gen = driver_chrome("https://steamdb.info/genres/","//div[@class='row']")
for g in cro_gen:
    g = g.text.replace(" ","_")
    tags_genre_query.append("genre_"+g+' INT(1)')

# driver_tags.quit
tag_genre_join = (", \n".join(tags_genre_query))

# SQL문 필터링
tag_genre_join = name_replace(tag_genre_join)
try: 
    cursor = db.cursor()
    SQL_QUERY = """
    # DROP DATABASE IF EXISTS 테이블이름;
    # CREATE DATABASE 테이블이름 DEFAULT CHARSET=utf8 COLLATE=utf8_bin;

    USE steam_test;
    DROP TABLE IF EXISTS steam_test.data_merge;
    CREATE TABLE steam_test.data_merge (
        appid int(7),
        name varchar(100),
        owners int(10),
        owners_variance int(7),
        negative int(7),
        positive int(6),
        score_rank int(3),
        ccu int(6),
        players_2weeks int(8),
        players_forever_variance int(7),
        players_2weeks_variance int(7),
        userscore int(2),
        median_forever int(3),
        players_forever int(7),
        price int(7),
        average_2weeks int(5),
        developer varchar(300),
        median_2weeks int(4),
        publisher varchar(100),
        average_forever int(6),
        tag_Dystopian_ int(1),
        tag_Parody_ int(1),
        tag_VR_Only int(1),
        """+tag_genre_join+""",
        PRIMARY KEY(appid)
        ) DEFAULT CHARSET=utf8 COLLATE=utf8_bin
    """
#     print(SQL_QUERY)
#     print("".join(SQL_QUERY))
    create_sql=("".join(SQL_QUERY))
    print(create_sql)
    cursor.execute(create_sql)
    db.commit()
finally:
    print("ok!")
    pass

#----------------------------------------------------------------------------------------    
end_time = time.time()
print("{} 초(처리시간)".format(end_time-start_time))

<class 'str'>
<class 'str'>

    # DROP DATABASE IF EXISTS 테이블이름;
    # CREATE DATABASE 테이블이름 DEFAULT CHARSET=utf8 COLLATE=utf8_bin;

    USE steam_test;
    DROP TABLE IF EXISTS steam_test.data_merge;
    CREATE TABLE steam_test.data_merge (
        appid int(7),
        name varchar(100),
        owners int(10),
        owners_variance int(7),
        negative int(7),
        positive int(6),
        score_rank int(3),
        ccu int(6),
        players_2weeks int(8),
        players_forever_variance int(7),
        players_2weeks_variance int(7),
        userscore int(2),
        median_forever int(3),
        players_forever int(7),
        price int(7),
        average_2weeks int(5),
        developer varchar(300),
        median_2weeks int(4),
        publisher varchar(100),
        average_forever int(6),
        tag_Dystopian_ int(1),
        tag_Parody_ int(1),
        tag_Indie INT(1), 
tag_Action_ INT(1), 
tag_Adventure INT(1), 
tag_Casual INT(1), 
tag_Simulation INT(1), 
t

ok!
18.04292058944702 초(처리시간)


### 초기 ID, name 데이터

In [5]:
# start_time = time.time()
# #-----------------------------------------------------------------------------------------
# with open("request_all.txt","r") as file:
#     f = file.read()
#     data = json.loads(f)
#     for k,v in data.items():
#         key = []
# #         print(k)
#         value = []
#         for k2,v2 in v.items():
#             print(k2," : ",v2)
#             if k2 == "score_rank":
#                 if v2 == "":
#                     v2 = 0
#             elif k2 == "developer" or k2 =="publisher":
#                 if v2 =="":
#                     v2 = "null"
#             elif k2 == "price":
#                 if v2 == "":
#                     v2 == 999999                    
#             key.append(k2)
#             value.append(v2)
            
#         key = tuple(key)
#         key = ", ".join(key)
#         value = tuple(value)
#         sql ="""
#             INSERT INTO steam_test.steamtable2 ({}) 
#             VALUES 
#             {};
#             """.format(key, value)
#         print(sql)
#         cursor.execute(sql)
#         db.commit()
# #----------------------------------------------------------------------------------------    
# end_time = time.time()
# print("{} 초(처리시간)".format(end_time-start_time))

### 최종데이터

In [8]:
start_time = time.time()
#-----------------------------------------------------------------------------------------

def name_replace(name):
    name = name.replace("&","and").replace(".","dot")\
    .replace("Mod","Mod_").replace("'","").replace("-","_").replace("Linear","Linear_")\
    .replace("Action","Action_").replace("Foreign","Foreign_").replace("All","All_")
    return name

def select_number(start, last):
    get_data = """
    # select appid, name from steamtable2 LIMIT 2
    SELECT appid, name
    FROM steamtable2
    WHERE
        appid BETWEEN {} AND {};
    """.format(start, last)
    
    cursor = db.cursor()
    cursor.execute(get_data)
    row_names = cursor.fetchall()

    count = 0
    for s_num in row_names:
        print(s_num[0],s_num[1],count)
        count += 1
#         if count % 1000 == 0:
#             print(s_num[0],s_num[1],count)
#             count += 1
#         else:
#             count += 1
        
        column_str=[]
        row_str=[]
        
        steamspy_url =('https://steamspy.com/api.php?request=appdetails&appid={}'.format(s_num[0]))
        appdetails_response = requests.get(steamspy_url)
        data = appdetails_response.json()
#         print(data)
        for data_k, data_v in data.items():
            data_k = name_replace(data_k).replace(" ","_")  
            if data_k == 'tags':
#                 print(data['tags'])
                if data['tags'] == []:
                    pass
                else:
                    for tag_k ,tag_v in data['tags'].items():
                        tag_k = name_replace(tag_k).replace(" ","_")
                        column_str.append(("tag_{}").format(tag_k))
                        row_str.append(1)
                        
            elif data_k == "score_rank":
                if data_k == "":
                    column_str.append(data_k)
                    row_str.append(0)
                    
            elif data_k == "developer" or data_k =="publisher":
                if data_v =="":
                    column_str.append(data_k)
                    row_str.append("null")
                    
            elif data_k == "price":
                if data_v == "":
                    column_str.append(data_k)
                    row_str.append(999999)
                    
            else:
                column_str.append(data_k)
                row_str.append(data_v)
#         print()
        try:
            cursor = db.cursor()
            sql = ("INSERT INTO steam_test.data_merge ("+(",".join(column_str)+") VALUES {}"\
                                                          .format(tuple(row_str))))
#             print(sql)
            cursor.execute(sql)
            db.commit()
        finally:
#             print("insert ok!")
            pass    
#----------------------------------------------------------------------------------------

# select_number(10 ,8339)
select_number(200001,500000)
#----------------------------------------------------------------------------------------    
end_time = time.time()
print("{} 초(처리시간)".format(end_time-start_time))

200010 Quantum Conundrum 0
200020 Global Ops: Commando Libya 1
200050 Naval War: Arctic Circle 2
200060 Sacred 2: Fallen Angel (EU) 3
200080 Nancy Drew: Shadow at the Water's Edge 4
200110 Nosgoth 5
200130 Puzzler World 2 6
200150 Battle Chess: Game of Kings 7
200170 Worms Revolution 8
200190 Sideway New York 9
200210 Realm of the Mad God 10
200230 Off-Road Drive 11
200260 Batman: Arkham City - Game of the Year Edition 12
200350 Starvoid 13
200370 A Game of Dwarves 14
200390 Oil Rush 15
200410 SOL: Exodus 16
200490 Memento Mori 17
200510 XCOM: Enemy Unknown 18
200550 Dungeons - The Dark Lord 19
200630 Gettysburg: Armored Warfare 20
200670 Disciples III - Resurrection 21
200710 Torchlight II 22
200900 Cave Story+ 23
200910 Before the Echo 24
200940 Sonic CD 25
200960 Geneforge 1 26
200980 Geneforge 2 27
200990 Geneforge 3 28
201000 Geneforge 4: Rebellion 29
201010 Geneforge 5: Overthrow 30
201020 NBA 2K12 31
201040 Galcon Legends 32
201070 Revelations 2012 33
201190 Magic: The Gathering

211440 Adventures of Shuggy 265
211500 RaceRoom Racing Experience 266
211580 Wanderlust: Rebirth 267
211600 Thief Gold 268
211740 Thief II: The Metal Age 269
211780 Conflict Desert Storm 270
211800 Dead Hungry Diner 271
211820 Starbound 272
211880 Bullet Run 273
211900 Conquest of Elysium 3 274
211970 Test Drive: Ferrari Racing Legends 275
212010 Galaxy on Fire 2 Full HD 276
212030 Kung Fu Strike - The Warrior's Rise 277
212050 Resonance 278
212070 Star Conflict 279
212110 Sugar Cube: Bittersweet Factory 280
212160 Vindictus 281
212180 Combat Arms 282
212200 Mabinogi 283
212220 Dungeon Fighter Online 284
212240 Atlantica 285
212370 Arctic Combat 286
212390 Continent of the Ninth Seal 287
212410 Inversion 288
212480 Sonic & All-Stars Racing Transformed 289
212500 The Lord of the Rings Online 290
212542 Red Orchestra 2 - Dedicated Server 291
212580 The Amazing Spider-Man 292
212630 Tom Clancy's Ghost Recon: Future Soldier 293
212680 FTL: Faster Than Light 294
212700 Party of Sin 295
2127

224600 Defiance 523
224700 The Game of Life 524
224740 Clockwork Empires 525
224760 FEZ 526
224820 Super House of Dead Ninjas 527
224860 Arma Tactics 528
224900 Iron Sky: Invasion 529
224920 Legacy of Kain: Soul Reaver 530
224940 Legacy of Kain: Soul Reaver 2 531
224960 Tomb Raider I 532
224980 Tomb Raider IV: The Last Revelation 533
225000 Tomb Raider V: Chronicles 534
225020 Tomb Raider VI: The Angel of Darkness 535
225080 Brothers - A Tale of Two Sons 536
225140 Duke Nukem 3D: Megaton Edition 537
225160 Shadow Warrior Classic Redux 538
225180 Narco Terror 539
225220 NASCAR The Game: 2013 540
225260 Brutal Legend 541
225280 Full Mojo Rampage 542
225300 Tomb Raider II 543
225320 Tomb Raider III 544
225360 DARK 545
225420 Cities in Motion 2 546
225540 Just Cause 3 547
225600 Blade Symphony 548
225640 Sacred 2 Gold 549
225760 Nexuiz STUPID Mode 550
225840 Sven Co-op 551
226100 PixelJunk Nom Nom Galaxy 552
226120 Weird Worlds: Return to Infinite Space 553
226320 Marvel Heroes Omega 554
2

238210 System Shock 2 787
238240 Edge of Space 788
238260 Pinball Arcade 789
238280 Legend of Dungeon 790
238320 Outlast 791
238370 Magicka 2 792
238430 Contagion 793
238460 BattleBlock Theater 794
238530 Super Puzzle Platformer Deluxe 795
238590 Loadout Campaign Beta 796
238630 Fist Puncher 797
238710 Substance Designer 4 798
238750 Might & Magic X - Legacy 799
238870 Citadels 800
238890 Skyward Collapse 801
238910 Bionic Dues 802
238930 7 Grand Steps: What Ancients Begat 803
238960 Path of Exile 804
239030 Papers, Please 805
239070 Hammerwatch 806
239090 Samurai Gunn 807
239120 FINAL FANTASY III 808
239140 Dying Light 809
239160 Thief 810
239200 Amnesia: A Machine for Pigs 811
239220 The Mighty Quest For Epic Loot 812
239250 Castlevania: Lords of Shadow 2 813
239350 Spelunky 814
239410 Catan: Creator's Edition 815
239430 Q.U.B.E: Director's Cut 816
239450 Gun Monkeys 817
239660 Soldier Front 2 818
239700 Hate Plus 819
239720 PAC-MAN and the Ghostly Adventures 820
239760 Football Mana

250700 Super Time Force Ultra 1062
250720 Starlight Inception 1063
250740 Ragnarok Online - Free to Play - European Version 1064
250760 Shovel Knight: Treasure Trove 1065
250820 SteamVR 1066
250900 The Binding of Isaac: Rebirth 1067
251020 Railroad X 1068
251060 Wargame: Red Dragon 1069
251110 INFRA 1070
251130 Chroma Squad 1071
251150 The Legend of Heroes: Trails in the Sky 1072
251170 Damned 1073
251210 Hive 1074
251230 Livelock 1075
251270 Corpse Party 1076
251290 The Legend of Heroes: Trails in the Sky SC 1077
251310 Eleusis 1078
251370 Escape Goat 1079
251390 Call of Cthulhu: The Wasted Land 1080
251410 Dark Matter 1081
251430 The Inner World 1082
251450 Heroes of Havoc: Idle Adventures 1083
251470 TowerFall Ascension 1084
251490 Gravity Ghost 1085
251510 Constant C 1086
251530 Anomaly Korea 1087
251570 7 Days to Die 1088
251590 Soul Saga 1089
251630 The Impossible Game 1090
251670 Battle Nations 1091
251690 Speedball 2 HD 1092
251710 Chainsaw Warrior 1093
251730 Legend of Grimroc

259960 YOU DON'T KNOW JACK Vol. 3 1323
259980 YOU DON'T KNOW JACK Vol. 4 The Ride 1324
260000 YOU DON'T KNOW JACK MOVIES 1325
260020 YOU DON'T KNOW JACK SPORTS 1326
260040 YOU DON'T KNOW JACK TELEVISION 1327
260060 YOU DON'T KNOW JACK HEADRUSH 1328
260130 Agarest: Generations of War Zero 1329
260160 The Last Tinker: City of Colors 1330
260190 Marc Eck's Getting Up: Contents Under Pressure 1331
260210 Assassins Creed Liberation HD 1332
260230 Valiant Hearts: The Great War / Soldats Inconnus : Mmoires de la Grande Guerre 1333
260250 Blood of the Werewolf 1334
260270 Shadows: Price For Our Sins Bonus Edition 1335
260330 Flockers 1336
260380 Ninja Cats vs Samurai Dogs 1337
260410 Get Off My Lawn! 1338
260430 The Four Kings Casino and Slots 1339
260510 World Basketball Tycoon 1340
260530 So Many Me 1341
260550 Haunted 1342
260570 Gray Matter 1343
260650 Cold War 1344
260690 Dark Fall: The Journal 1345
260710 Dark Fall 2: Lights Out 1346
260730 Desperados: Wanted Dead or Alive 1347
260750 Ne

268830 Doctor Who: The Adventure Games 1575
268850 EVGA Precision XOC 1576
268870 Satellite Reign 1577
268890 Masters of the World - Geopolitical Simulator 3 1578
268910 Cuphead 1579
268930 Malevolence: The Sword of Ahkranox 1580
268970 You Are Not The Hero 1581
268990 The Dishwasher: Vampire Smile 1582
269010 Science Girls 1583
269030 The Yawhg 1584
269050 Year Walk 1585
269110 Super Motherload 1586
269150 Luxuria Superbia 1587
269170 Sports Bar VR 1588
269210 Hero Siege 1589
269230 Aces Wild: Manic Brawling Action! 1590
269250 WORLD END ECONOMiCA episode.01 1591
269270 LOVE 1592
269290 liteCam Game: 100 FPS Game Capture 1593
269310 Infectonator: Survivors 1594
269330 Chronology 1595
269350 Montas 1596
269370 Reaper - Tale of a Pale Swordsman 1597
269390 Dethroned! 1598
269430 Burning Cars 1599
269450 MindTex 1600
269470 MorphVOX Pro - Voice Changer 1601
269490 Bardbarian 1602
269530 Voice of Pripyat 1603
269550 Zoom Player 13 : Steam Edition 1604
269570 GTGD S1: More Than a Gamer 160

InternalError: (1054, "Unknown column 'tag_VR_Only' in 'field list'")

In [ ]:
start_time = time.time()
#-----------------------------------------------------------------------------------------

while count < 571:
    
    column_str=[]
    row_str=[]
    
    steamspy_url =('https://steamspy.com/api.php?request=appdetails&appid={}'.format(count))
    appdetails_response = requests.get(steamspy_url)
    data = appdetails_response.json()

    for data_k, data_v in data.items():
        data_k = name_replace(data_k).replace(" ","_")  

        if data_k == 'tags':
            for tag_k ,tag_v in data['tags'].items():
                tag_k = name_replace(tag_k).replace(" ","_")  
                column_str.append(("tag_{}").format(tag_k))
                row_str.append(1)
        else:
            column_str.append(data_k)
            row_str.append(data_v)

    print()
    try:
        cursor = db.cursor()
        sql = ("INSERT INTO steam_test.data_merge ("+(",".join(column_str)+") VALUES {}"\
                                                      .format(tuple(row_str))))
        print(sql)
        cursor.execute(sql)
        db.commit()
    finally:
        print("insert ok!")
        pass
                
    count += 10

#----------------------------------------------------------------------------------------    
end_time = time.time()
print("{} 초(처리시간)".format(end_time-start_time))

NameError: name 'time' is not defined